In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB 435.7 kB/s eta 0:00:34
   ---------------------------------------- 0.1/14.5 MB 655.4 kB/s eta 0:00:23
    --------------------------------------- 0.2/14.5 MB 1.1 MB/s eta 0:00:13
   - -------------------------------------- 0.4/14.5 MB 1.8 MB/s eta 0:00:08
   - -------------------------------------- 0.6/14.5 MB 2.4 MB/s eta 0:00:06
   -- ------------------------------------- 0.8/14.5 MB 2.6 MB/s eta 0:00:06
   -- ------------------------------------- 1.1/14.5 MB 2.9 MB/s eta 0:00:05
   --- ------------------------------------ 1.2/14.5 MB 3.2 MB/s eta 0:00:05
   ---- ----------------------------------- 1.5/14.5 MB

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings  #An embedding is a vector (list) of floating point numbers. The distance between two vectors measures their relatedness. Small distances suggest high relatedness and large distances suggest low relatedness.
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [23]:
mykey1 = ""

In [4]:
pdfreader = PdfReader('ATT.pdf')

In [5]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [6]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [7]:
len(texts)

67

In [8]:
texts

['Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best',
 'illia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also con

In [11]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings(openai_api_key=mykey1)

In [12]:
document_search = FAISS.from_texts(texts, embeddings)

In [13]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [15]:
chain = load_qa_chain(OpenAI(openai_api_key=mykey1), chain_type="stuff")

In [16]:
query = "transformers"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

C:\Users\HP\anaconda3\envs\testingopenai\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\nThe Transformer is the first transduction model relying entirely on self-attention to compute representations of its input and output without using sequence-aligned RNNs or convolution. It is a simple network architecture that uses attention mechanisms to model dependencies between input and output sequences, allowing for more parallelization and achieving superior translation quality in less time compared to other models.'

In [17]:
from langchain.document_loaders import OnlinePDFLoader

In [18]:
loader = OnlinePDFLoader("https://arxiv.org/pdf/1706.03762.pdf")

In [19]:
!pip install unstructured

     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ----- -------------------------------- 153.6/981.5 kB 4.6 MB/s eta 0:00:01
     -------------- ----------------------- 378.9/981.5 kB 3.9 MB/s eta 0:00:01
     ------------------------ ------------- 645.1/981.5 kB 4.5 MB/s eta 0:00:01
     ------------------------------------ - 931.8/981.5 kB 4.9 MB/s eta 0:00:01
     -------------------------------------  972.8/981.5 kB 5.1 MB/s eta 0:00:01
     -------------------------------------  972.8/981.5 kB 5.1 MB/s eta 0:00:01
     -------------------------------------  972.8/981.5 kB 5.1 MB/s eta 0:00:01
     -------------------------------------- 981.5/981.5 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ------------- -------------------------- 0.6/1.8 MB

In [20]:
data = loader.load()

ModuleNotFoundError: No module named 'pdf2image'

In [21]:
embeddings = OpenAIEmbeddings(openai_api_key=mykey1)

In [ ]:
!pip install chromadb

In [22]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
query = "Explain me about Attention is all you need"
index.query(query)
